In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

In [ ]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "2g"))
sc = SparkContext(conf=conf)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc).builder.appName("TP_ML_Email_spam").getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.csv('emails.csv',header=True,sep=",",inferSchema=True)

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
df.show(3)

In [ ]:
df.select('spam').show()

In [ ]:
df.filter(df['spam']=="1").count()

In [ ]:
df.filter(df['spam']=="0").count()

In [ ]:
df2 = df.filter((df['spam']=="1")|(df['spam']=="0"))

In [ ]:
df2.count()

In [ ]:
df2.groupBy('spam').count().show()

In [ ]:
txt1 = df2.select('text').first()

In [ ]:
type(txt1)

In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

In [ ]:
tokenized = tokenizer.transform(df2)

In [ ]:
tokenized.show(2)

In [ ]:
regexTokenized = regexTokenizer.transform(df2)# alternatively, pattern="\\w+", gaps(False)

In [ ]:
regexTokenized.show(2)

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [ ]:
data = remover.transform(regexTokenized)

In [ ]:
data.show(2)

In [ ]:
data.select('words').first()

In [ ]:
data.select('filtered').first()

In [ ]:
def mots(txt):
    return [w for w in txt if ((len(w) > 2) & (w != 'subject'))  ]

In [ ]:
str = ["subject","sam","de","fat","*","--"]
print(mots(str))

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
mots = udf(mots,ArrayType(StringType()))

In [ ]:
toNum = lambda str:int(str)

In [ ]:
toNum('3')+2

In [ ]:
toNum = udf(toNum,'int')

In [ ]:
data2 = data.withColumn('motsPur', mots(data["filtered"]))

In [ ]:
data2 = data2.withColumn('label', toNum(data["spam"]))

In [ ]:
data2.select('motsPur').first()

In [ ]:
data2.select('label').show(5)

In [ ]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="motsPur", outputCol="features")
data3 = cv.fit(data2).transform(data2)

In [ ]:
#data3.select('features').first()

In [ ]:
v = Row(

In [ ]:
train, test = data3.select('features','label').randomSplit([0.7,0.3],seed=100)

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
# train the model
model = nb.fit(train)

In [ ]:
# select example rows to display.
predictions = model.transform(test)
#predictions.show()

In [ ]:
# select example rows to display.
test2 = test.select('features')
predictions2 = model.transform(test2)
#predictions2.show()

In [ ]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")


In [ ]:
accuracy = evaluator.evaluate(predictions)


In [ ]:
accuracy

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
from pyspark.sql import Row
eml = spark.createDataFrame([
    Row(text="unbelievable new homes made easy  im wanting to show you this  homeowner  you have been pre - approved for a $ 454 , 169 home loan at a 3 . 72 fixed rate .  this offer is being extended to you unconditionally and your credit is in no way a factor .  to take advantage of this limited time opportunity  all we ask is that you visit our website and complete  the 1 minute post approval form  look foward to hearing from you ,  dorcas pittman",label=0),
    Row(text="localized software , all languages available .  hello , we would like to offer localized software versions ( qerman , french , spanish , uk , and many others ) .  all iisted software is availabie for immediate download !  no need to wait 2 - 3 week for cd deiivery !  just few exampies :  - norton lnternet security pro 2005 - $ 29 . 95  - windows xp professional with sp 2 fuil version - $ 59 . 95  - corei draw graphics suite 12 - $ 49 . 95  - dreamweaver mx 2004 ( homesite 5 . 5 including ) - $ 39 . 95  - macromedia studio mx 2004 - $ 119 . 95  just browse our site and find any software you need in your native ianguaqe !  best reqards ,  lauraiee",label=0)
    ])

In [ ]:
dfml = spark.read.csv('emailsTest.csv',header=True,sep=",",inferSchema=True)

In [ ]:
eml.show()

In [ ]:
eml.printSchema()

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
eml2 = regexTokenizer.transform(eml)
dfml = regexTokenizer.transform(dfml)

In [ ]:
remover2 = StopWordsRemover(inputCol="words", outputCol="filtered")
eml3 = remover2.transform(eml2)
dfml = remover2.transform(dfml)

In [ ]:
eml4 = eml3.withColumn('motsPur', mots(eml3["filtered"]))
dfml = dfml.withColumn('motsPur', mots(dfml["filtered"]))

In [ ]:
# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="motsPur", outputCol="features")
eml5 = cv.fit(eml4).transform(eml4)
dfml = cv.fit(dfml).transform(dfml)

In [ ]:
dfml.show()

In [ ]:
eml5.show()

In [ ]:
# select example rows to display.
eml6 =eml5.select('features','label')
tt = dfml.select('features')
predictions3 = model.transform(eml6)
predTest = model.transform(tt)



In [ ]:
tt.show(5)

In [ ]:
eml6.first()

In [ ]:
type(predictions3)

In [ ]:
predictions3.printSchema()

In [ ]:
predTest.printSchema()

In [ ]:
predictions3.count()

In [ ]:
predTest.select('prediction').toDF()

In [ ]:
predictions3.select('prediction').show(1)